In [16]:
using AdvancedHMC, Distributions, ForwardDiff, LinearAlgebra, Random

D = 30; initial_θ = zeros(D)
make_sym(A) = (transpose(A) + A) / 2.
sig = Matrix{Float64}(I, D, D)
sig_inv = make_sym(pinv(sig))

kernel = ones(D, D)
data = Random.rand(D)
data_errors = abs.(Random.rand(D))
data_errors = cat(data_errors...; dims=(1,2))

# Choose parameter dimentionality and initial parameter value
 
# distr = MvNormal(zeros(D), sig)

# Define the target distribution
# ℓπ = let distr=distr
#     θ -> logpdf(distr, θ)
# end

ℓπ = let D=D, sig_inv=sig_inv, data=data, data_errors=data_errors, kernel=kernel
        phi -> begin
            mu = zeros(D)
            pr_log = -1/2 * transpose(phi - mu) * sig_inv * (phi - mu)
            mu_ = kernel * phi
            li_log = -1/2 * transpose(data - mu_) * make_sym(pinv(data_errors)) * (data - mu_)
        return pr_log + li_log
        end
    end

# function ℓπ(phi)
#     mu = zeros(D)
#     return exp(-1/2 * transpose(phi - mu) * sig_inv * (phi - mu))
# end

# Set the number of samples to draw and warmup iterations
n_samples, n_adapts = 1_000, 1_00 

# Define a Hamiltonian system
metric = DiagEuclideanMetric(D)
hamiltonian = Hamiltonian(metric, ℓπ, ForwardDiff)  

# Define a leapfrog solver, with initial step size chosen heuristically
initial_ϵ = find_good_eps(hamiltonian, initial_θ) 
integrator = Leapfrog(initial_ϵ)

# Define an HMC sampler, with the following components
#   - multinomial sampling scheme,
#   - generalised No-U-Turn criteria, and
#   - windowed adaption for step-size and diagonal mass matrix
proposal = NUTS{MultinomialTS, GeneralisedNoUTurn}(integrator)
adaptor = StanHMCAdaptor(Preconditioner(metric), NesterovDualAveraging(0.8, integrator))

# Run the sampler to draw samples from the specified Gaussian, where
#   - `samples` will store the samples
#   - `stats` will store diagnostic statistics for each sample
samples, stats = sample(hamiltonian, proposal, initial_θ, n_samples, adaptor, n_adapts; progress=false);

┌ Info: Finished 100 adapation steps
│   adaptor = StanHMCAdaptor(
    pc=DiagPreconditioner,
    ssa=NesterovDualAveraging(γ=0.05, t_0=10.0, κ=0.75, δ=0.8, state.ϵ=0.018324872972971144),
    init_buffer=75, term_buffer=50, window_size=25,
    state=window(76, 50), window_splits()
)
│   τ.integrator = Leapfrog(ϵ=0.0183)
│   h.metric = DiagEuclideanMetric([1.0, 1.0, 1.0, 1.0, 1.0, 1 ...])
└ @ AdvancedHMC /Users/ta_nyan/.julia/packages/AdvancedHMC/Sn9Ek/src/sampler.jl:159
┌ Info: Finished 1000 sampling steps for 1 chains in 139.959119357 (s)
│   h = Hamiltonian(metric=DiagEuclideanMetric([1.0, 1.0, 1.0, 1.0, 1.0, 1 ...]))
│   τ = NUTS{MultinomialTS,Generalised}(integrator=Leapfrog(ϵ=0.0183), max_depth=10), Δ_max=1000.0)
│   EBFMI_est = 1.0357390818844137
│   average_acceptance_rate = 0.9129499329345763
└ @ AdvancedHMC /Users/ta_nyan/.julia/packages/AdvancedHMC/Sn9Ek/src/sampler.jl:178


In [17]:
using Plots
plotly()


Plots.PlotlyBackend()

In [18]:
coeff = [[samples[j][i] for j in range(1, stop=n_samples)] for i in eachindex(samples[1])];

In [19]:
histogram(coeff[5])

<!DOCTYPE html>
 
 
 Plots.jl